In [82]:
import os
import pandas as pd
from data_reader import *
from models import *
from tf_idf import *
from colorama import Fore
colorama.init(autoreset=True)

In [83]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), ".."))
file_path = os.path.join(parent_directory, "dataset/processed-full-name-dup.csv")
data = pd.read_csv(file_path)

parent_directory = os.path.abspath(os.path.join(os.getcwd(), ".."))
file_path = os.path.join(parent_directory, "dataset/dsai-k66-gender-annotated-name.csv")
test_data = pd.read_csv(file_path)

In [84]:
print(data.head())
print("--------------------")
print(test_data.head())

           Full_Name  Gender
0      ngô xuân tùng       1
1  bùi dương thảo vy       0
2        lưu thế huy       1
3     nguyễn thị vân       0
4    dương minh long       1
--------------------
         Full_Name  Gender
0  Nguyễn Thanh An       1
1    Nguyễn Thế An       1
2        Lê Bá Anh       1
3       Vũ Lâm Anh       1
4     Ngô Việt Anh       1


In [85]:
RANDOM_STATE = 1989

In [86]:
# Add new name
X_test = []
y_test = []

In [87]:
# Use DSAI students' names
X_test = test_data["Full_Name"].values
y_test = test_data["Gender"].values

In [88]:
X = data["Full_Name"].values
y = data["Gender"].values

# Train test split
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
print(X_train)
print("--------------------")

# TF-IDF Tokenizer
TF_IDF = Compute_TF_IDF(X_train)
tfidf_X_train = TF_IDF.compute_tf_idf()
tfidf_X_test = TF_IDF.compute_tf_idf_for_test(X_test)
print(tfidf_X_train)
print("Shape:", tfidf_X_train.shape)
print("--------------------")

truncated_tfidf_X_train, truncated_tfidf_X_test = truncated_svd(tfidf_X_train, tfidf_X_test, random_state=RANDOM_STATE)
print(truncated_tfidf_X_train)
print("Shape:", truncated_tfidf_X_train.shape)

['nguyễn phú quý' 'nguyễn hoàng tuấn' 'nguyễn phương mai' ...
 'nguyễn ngọc thành thật' 'võ minh vũ' 'nguyễn tấn ngà']
--------------------


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Shape: (21098, 1089)
--------------------
[[ 1.21496892  1.16546849 -1.04168118 ... -0.04638662  0.68607589
   1.91595948]
 [ 1.73901523  2.28288229 -0.40990857 ...  0.05136551  0.14451796
  -0.02507637]
 [ 1.69962091  0.74757005 -1.00416025 ... -0.05767348 -0.02964623
  -0.24723676]
 ...
 [ 2.01781562  1.32630302 -1.05541287 ...  0.04573085 -0.19647034
   0.06226329]
 [ 1.11441369  1.21780091  1.0447772  ...  0.07127101  0.05719485
   0.06819658]
 [ 1.2172349   1.20825693 -1.08446803 ... -0.18646707 -0.02090141
  -0.09960819]]
Shape: (21098, 100)


In [89]:
logreg = MyLogisticRegression(truncated_tfidf_X_train, truncated_tfidf_X_test, y_train, y_test, random_state=RANDOM_STATE,
                              regularization="lasso", regularization_strength=0.01)
logreg.fit()

LOGISTIC REGRESSION


In [90]:
y_pred = logreg.predict()
convert_dict = {0: "Female", 1: "Male"}

print("{0:30}{1:10}{2:10}".format("Name", "True", "Predicted"))
print("{0:30}{1:10}{2:10}".format("----------", "--------", "--------"))
for i, name in enumerate(X_test):
    if y_test[i] == y_pred[i]: 
        print("{0:30}{1:10}{2:10}".format(name, convert_dict[y_test[i]], convert_dict[y_pred[i]]))
    else:
        print("{0:30}{1:10}{2:10}{3:^11}".format(name, convert_dict[y_test[i]], convert_dict[y_pred[i]], "ERROR"))

Name                          True      Predicted 
----------                    --------  --------  
Nguyễn Thanh An               Male      Male      
Nguyễn Thế An                 Male      Male      
Lê Bá Anh                     Male      Male      
Vũ Lâm Anh                    Male      Male      
Ngô Việt Anh                  Male      Male      
Phan Công Anh                 Male      Male      
Hồ Ngọc Ánh                   Female    Female    
Trương Gia Bách               Male      Male      
Nguyễn Đình Bắc               Male      Male      
Lê Ngọc Bình                  Male      Female       ERROR   
Nguyễn Thanh Bình             Male      Male      
Đậu Văn Cần                   Male      Male      
Nguyễn Hữu Chí                Male      Male      
Trần Dương Chính              Male      Male      
Đoàn Ngọc Cường               Male      Male      
Nguyễn Minh Cường             Male      Male      
Nguyễn Tiến Doanh             Male      Male      
Bùi Tiến Dũng       

In [91]:
bnb = MyBernoulliNaiveBayes(tfidf_X_train, tfidf_X_test, y_train, y_test)
bnb.fit()

BERNOULLI NAIVE BAYES

In [92]:
y_pred = bnb.predict()
convert_dict = {0: "Female", 1: "Male"}

print("{0:30}{1:10}{2:10}".format("Name", "True", "Predicted"))
print("{0:30}{1:10}{2:10}".format("----------", "--------", "--------"))
for i, name in enumerate(X_test):
    if y_test[i] == y_pred[i]: 
        print("{0:30}{1:10}{2:10}".format(name, convert_dict[y_test[i]], convert_dict[y_pred[i]]))
    else:
        print("{0:30}{1:10}{2:10}{3:^11}".format(name, convert_dict[y_test[i]], convert_dict[y_pred[i]], "ERROR"))

Name                          True      Predicted 
----------                    --------  --------  
Nguyễn Thanh An               Male      Male      
Nguyễn Thế An                 Male      Male      
Lê Bá Anh                     Male      Male      
Vũ Lâm Anh                    Male      Male      
Ngô Việt Anh                  Male      Male      
Phan Công Anh                 Male      Male      
Hồ Ngọc Ánh                   Female    Female    
Trương Gia Bách               Male      Male      
Nguyễn Đình Bắc               Male      Male      
Lê Ngọc Bình                  Male      Male      
Nguyễn Thanh Bình             Male      Male      
Đậu Văn Cần                   Male      Male      
Nguyễn Hữu Chí                Male      Male      
Trần Dương Chính              Male      Male      
Đoàn Ngọc Cường               Male      Male      
Nguyễn Minh Cường             Male      Male      
Nguyễn Tiến Doanh             Male      Male      
Bùi Tiến Dũng                 M